In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls datasets/CTI-Data/

cti.cod  cti.nav  cti-stats.txt  cti.std  cti.tra  readme.txt


# action
- user id
- page: page id and split /
- request type?

In [14]:
pages = pd.read_csv('./datasets/CTI-Data/cti.cod', sep='\t', header=None, names=['id', 'url'])

In [15]:
pages.head()

,id,url
0,0,/admissions/
1,1,/admissions/career.asp
2,2,/admissions/checklist.asp
3,3,/admissions/costs.asp
4,4,/admissions/default.asp


In [36]:
def getUrlSections(row):
    url = row['url']
    idx = url.find("?")
    if idx != -1:
        url = url[:idx]
    url_paths = url.split("/")[1:6]
    if len(url_paths) < 5:
        url_paths += ['']* (5-len(url_paths))
    return url_paths
    
    

In [37]:
url_sections = [getUrlSections(pages.iloc[i,:]) for i in range(len(pages))]

In [39]:
url_sections = np.array(url_sections)

In [40]:
url_sections.shape

(683, 5)

In [46]:
for i in range(5):
    col_name = f'url{i+1}'
    pages[col_name] = url_sections[:, i]
    pages[col_name] = pages[col_name].astype('category')
    pages[f'{col_name}_c'] = pages[col_name].cat.codes


In [48]:
pages.head(20)

,id,url,url1,url2,url3,url4,url5,url1_c,url2_c,url3_c,url4_c,url5_c
0,0,/admissions/,admissions,,,,,0,0,0,0,0
1,1,/admissions/career.asp,admissions,career.asp,,,,0,12,0,0,0
2,2,/admissions/checklist.asp,admissions,checklist.asp,,,,0,14,0,0,0
3,3,/admissions/costs.asp,admissions,costs.asp,,,,0,17,0,0,0
4,4,/admissions/default.asp,admissions,default.asp,,,,0,22,0,0,0
5,5,/admissions/general.asp,admissions,general.asp,,,,0,36,0,0,0
6,6,/admissions/helloworld/arabic.asp,admissions,helloworld,arabic.asp,,,0,41,7,0,0
7,7,/admissions/helloworld/chinese.asp,admissions,helloworld,chinese.asp,,,0,41,21,0,0
8,8,/admissions/helloworld/italian.asp,admissions,helloworld,italian.asp,,,0,41,45,0,0
9,9,/admissions/helloworld/portugese.asp,admissions,helloworld,portugese.asp,,,0,41,58,0,0


# Read sessions

In [157]:
!head ./datasets/CTI-Data/cti.nav

In [60]:
pages[pages.url == '/courses/syllabus.asp?course=211-21-504&q=2&y=2002&id=123'][['id', 'url1_c', 'url2_c', 'url3_c', 'url4_c', 'url5_c']].values

array([], shape=(0, 6), dtype=int64)

In [58]:
sessions = []
with open('./datasets/CTI-Data/cti.nav', 'r') as f:
    l = ""
    header = f.readline()
    user_id = int(header[header.find("=")+1: header.find(")")])
    session = []
    l = f.readline()
    while f and not l.startswith("---"):
        print(l)
        url = l.split("\t")[1]
        page_row = pages[pages.url == url]
        session.append(page_row[['id', 'url1_c', 'url2_c', 'url3_c', 'url4_c', 'url5_c']].values)
        l = f.readline()

10203611	/search/newsearch.asp	/news/default.asp

10203615	/search/newsearch.asp	/search/newsearch.asp

10203624	/search/newsearch.asp	/search/newsearch.asp

10203628	/search/newsearch.asp	/search/newsearch.asp

10203630	/programs/	/search/newsearch.asp

10203632	/programs/2002/gradcs2002.asp	/programs/

10203639	/programs/courses.asp?depcode=21&deptmne=csc&courseid=211	/programs/2002/gradcs2002.asp

10203648	/courses/syllabus.asp?course=211-21-502&q=2&y=2002&id=483	/programs/courses.asp?depcode=21&deptmne=csc&courseid=211

10203658	/courses/syllabus.asp?course=211-21-503&q=2&y=2002&id=363	/programs/courses.asp?depcode=21&deptmne=csc&courseid=211

10203743	/courses/syllabus.asp?course=211-21-503&q=2&y=2002&id=363	/programs/courses.asp?depcode=21&deptmne=csc&courseid=211

10203746	/programs/courses.asp?depcode=21&deptmne=csc&courseid=211	/programs/2002/gradcs2002.asp

10203748	/courses/syllabus.asp?course=211-21-504&q=2&y=2002&id=123	/programs/courses.asp?depcode=21&deptmne=csc&courseid

In [74]:
pages[pages.url.str.startswith("/courses/syllabus.asp?course=211-21")]

,id,url,url1,url2,url3,url4,url5,url1_c,url2_c,url3_c,url4_c,url5_c
74,74,/courses/syllabus.asp?course=211-21-301&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
75,75,/courses/syllabus.asp?course=211-21-601&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
76,76,/courses/syllabus.asp?course=211-21-603&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
77,77,/courses/syllabus.asp?course=211-21-604&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
78,78,/courses/syllabus.asp?course=211-21-605&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
79,79,/courses/syllabus.asp?course=211-21-901&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
80,80,/courses/syllabus.asp?course=211-21-902&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
81,81,/courses/syllabus.asp?course=211-21-903&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
82,82,/courses/syllabus.asp?course=211-21-904&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0
83,83,/courses/syllabus.asp?course=211-21-905&q=3&y=...,courses,syllabus.asp,,,,4,80,0,0,0


In [69]:
session

[array([[679,  14,  56,   0,   0,   0]]),
 array([[679,  14,  56,   0,   0,   0]]),
 array([[679,  14,  56,   0,   0,   0]]),
 array([[679,  14,  56,   0,   0,   0]]),
 array([[574,  11,   0,   0,   0,   0]]),
 array([[585,  11,   2,  34,   0,   0]]),
 array([[604,  11,  18,   0,   0,   0]]),
 array([], shape=(0, 6), dtype=int64),
 array([], shape=(0, 6), dtype=int64),
 array([], shape=(0, 6), dtype=int64),
 array([[604,  11,  18,   0,   0,   0]]),
 array([], shape=(0, 6), dtype=int64)]

In [108]:
urls = set()
i = 0
with open('./datasets/CTI-Data/cti.nav', 'r') as f:
    l = f.readline()
    while f and l:
        if not (l.startswith("--") or l.startswith("SESSION")):
            url = (l.split("\t")[1])
            idx = url.find("?")
            if idx != -1:
                query = url[idx+1:]
                queryParams = query.split("&")
                queryParams = sorted([query.split("=")[0] for query in queryParams])
                queryParams = "&".join(queryParams)
                url = url[:idx] + "/" + queryParams
            #print("URL:", url)
            urls.add(url)

        l = f.readline()

In [109]:
len(urls)

668

In [110]:
len(pages)

683

In [111]:
urls = pd.DataFrame(sorted(urls), columns=['url'])
urls['url'] = urls['url'].astype('category')
urls['url_c'] = urls['url'].cat.codes

In [112]:
urls.head()

,url,url_c
0,/admissions/,0
1,/admissions/_vti_cnf/general.asp,1
2,/admissions/_vti_cnf/requirements.asp,2
3,/admissions/aboutgrad.asp,3
4,/admissions/career.asp,4


In [113]:
url_sections = [getUrlSections(pages.iloc[i,:]) for i in range(len(urls))]
url_sections = np.array(url_sections)
for i in range(5):
    col_name = f'url{i+1}'
    urls[col_name] = url_sections[:, i]
    urls[col_name] = urls[col_name].astype('category')
    urls[f'{col_name}_c'] = urls[col_name].cat.codes

In [156]:
urls.iloc[30:40, :]

,url,url_c,url1,url1_c,url2,url2_c,url3,url3_c,url4,url4_c,url5,url5_c
30,/advising/dars.asp,30,advising,1,graduation.asp,36,,0,,0,,0
31,/advising/dars.asp/section,31,advising,1,graduation.asp,36,,0,,0,,0
32,/advising/default.asp,32,advising,1,graduation.asp,36,,0,,0,,0
33,/advising/faculty.asp,33,advising,1,java,45,graduate.asp,42,,0,,0
34,/advising/faq_program.asp,34,advising,1,java,45,overview.asp,55,,0,,0
35,/advising/grad_scholarships.asp,35,advising,1,java,45,undergraduate.asp,71,,0,,0
36,/advising/grad_scholarships.asp/section,36,advising,1,nsf_scholarships.asp,52,,0,,0,,0
37,/advising/graduation.asp,37,advising,1,nsf_scholarships.asp,52,,0,,0,,0
38,/advising/graduation.asp/section,38,advising,1,policies.asp,57,,0,,0,,0
39,/advising/hci/jobs.html,39,advising,1,scholarship_finder.asp,62,,0,,0,,0


In [143]:
sessions = []
with open('./datasets/CTI-Data/cti.nav', 'r') as f:
    l = ""
    header = f.readline()
    while header:
        user_id = int(header[header.find("=")+1: header.find(")")])
        session = []
        l = f.readline()
        while f and not l.startswith("---"):
            #print(l)
            url = l.split("\t")[1]
            idx = url.find("?")
            if idx != -1:
                query = url[idx+1:]
                queryParams = query.split("&")
                queryParams = sorted([query.split("=")[0] for query in queryParams])
                queryParams = "&".join(queryParams)
                url = url[:idx] + "/" + queryParams
            page_row = urls[urls.url == url]
            session.append(page_row[['url_c', 'url1_c', 'url2_c', 'url3_c', 'url4_c', 'url5_c']].values)
            l = f.readline()
        if len(session) > 10:
            sessions.append(session)
        header = f.readline()

In [144]:
len(sessions)

2326

In [145]:
sessions_len = [len(session) for session in sessions]

In [146]:
max(sessions_len), min(sessions_len)

(422, 11)

In [147]:
sessions_len.index(422)

1730

In [149]:
urls.to_csv("urls.csv", index=False)

In [150]:
import pickle

In [152]:
pickle.dump(sessions, open("./datasets/CTI-Data/cti_sessions.p", "wb"))

In [154]:
sessions = pickle.load( open( "./datasets/CTI-Data/cti_sessions.p", "rb" ) )

In [155]:
len(sessions)

2326